# Torch Autograd

torch.autograd - движок автоматического дифференцирования PyTorch, который обеспечивает обучение нейронных сетей.

В этом разделе находится концептуальное представление о том, как autograd помогает нейронной сети обучаться.

## Справка

Нейронные сети (НС) представляют собой набор вложенных функций, которые выполняются на некоторых входных данных. Эти функции определяются параметрами (состоящими из весов и смещений), которые в PyTorch хранятся в тензорах.

Обучение NN происходит в два этапа:

Прямое распространение (Forward Propagation): При прямом распространении NN делает свое лучшее предположение о правильном выходе. Для этого он прогоняет входные данные через каждую из своих функций.

Обратное распространение: При обратном распространении NN корректирует свои параметры пропорционально ошибке в своем предположении. Для этого он проходит назад от выхода, собирает производные ошибки по параметрам функций (градиенты) и оптимизирует параметры с помощью градиентного спуска. Для более подробного описания backprop посмотрите [это видео от 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Использование в PyTorch

Давайте рассмотрим один шаг обучения. Для этого примера мы загрузим предварительно обученную модель resnet18 из torchvision. Мы создаем тензор случайных данных для представления одного изображения с 3 каналами, высотой и шириной 64, и соответствующую метку, инициализированную некоторыми случайными значениями. Метка в предварительно обученных моделях имеет форму (1,1000).

Примечание: это руководство работает только на CPU и не сможет сработать на GPU (даже если тензоры переместить на GPU)

In [2]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Далее мы прогоняем входные данные через модель по всем ее слоям, чтобы сделать предсказание. Это прямой проход

In [3]:
prediction = model(data)  # forward pass

Мы используем предсказание модели и соответствующую метку для вычисления ошибки (loss). Следующим шагом будет обратное распространение этой ошибки по сети. Обратное распространение запускается, когда мы вызываем .backward() на тензоре ошибок. Затем Autograd вычисляет и сохраняет градиенты для каждого параметра модели в атрибуте .grad параметра.

In [4]:
loss = (prediction - labels).sum()
loss.backward()  # backward pass

Далее мы загружаем оптимизатор, в данном случае SGD со скоростью обучения 0.01 и импульсом 0.9. Мы регистрируем все параметры модели в оптимизаторе

In [5]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Наконец, мы вызываем .step(), чтобы запустить градиентный спуск. Оптимизатор корректирует каждый параметр по его градиенту, хранящемуся в .grad

In [6]:
optim.step()  # gradient descent

На данный момент у нас есть все необходимое для обучения нейронной сети